In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
# path to project directory
path = Path('/home/jupyter/fpl-prediction')

In [3]:
# read in training dataset
# created using fpl_data_clean notebook
train_df = pd.read_csv(path/'data/train.csv', index_col=0, dtype={'season':str})

In [4]:
train_df.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
77805,Grant_Hanley,18,2,90,Norwich,Wolverhampton Wanderers,0.295212,0.746215,True,0,1920,0.50
77806,Roberto_Pereyra,18,3,13,Watford,Manchester United,0.433797,1.560524,True,1,1920,0.75
77807,Sead_Kolasinac,18,2,0,Arsenal,Everton,1.413491,1.033814,False,0,1920,0.75
77808,Stuart_Armstrong,18,3,75,Southampton,Aston Villa,0.469252,0.522434,False,2,1920,1.00
77809,Rúben Diogo_da Silva Neves,18,3,90,Wolverhampton Wanderers,Norwich,0.746215,0.295212,False,2,1920,1.00


In [21]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77290 entries, 0 to 77289
Data columns (total 12 columns):
player                                 77290 non-null object
gw                                     77290 non-null int64
position                               77290 non-null int64
minutes                                77290 non-null int64
team                                   77290 non-null object
opponent_team                          77290 non-null object
relative_market_value_team             77290 non-null float64
relative_market_value_opponent_team    77290 non-null float64
was_home                               77290 non-null bool
total_points                           77290 non-null int64
season                                 77290 non-null object
play_proba                             9289 non-null float64
dtypes: bool(1), float64(3), int64(4), object(4)
memory usage: 7.1+ MB


In [38]:
X, y = train_df.iloc[:,[1,2,3,4,5,6,7,8,10,11]], train_df.iloc[:,9]

In [39]:
# set categorical and continuous variables
cat_vars = ['gw', 'position', 'team', 'opponent_team', 'was_home', 'season']
cont_vars = ['minutes', 'relative_market_value_team', 'relative_market_value_opponent_team']

In [52]:
# encode string input values as integers
encoded_x = None
for var in cat_vars:
    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(X.loc[:,var])
    feature = feature.reshape(X.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
    feature = onehot_encoder.fit_transform(feature)
    if encoded_x is None:
        encoded_x = feature
    else:
        encoded_x = np.concatenate((encoded_x, feature), axis=1)
print("X shape: : ", encoded_x.shape)

X shape: :  (77290, 106)


In [56]:
X

,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,season,play_proba
0,1,2,0,West Ham United,Chelsea,0.895471,2.243698,False,1617,NaN
1,1,3,15,Everton,Tottenham Hotspur,1.057509,1.433690,True,1617,NaN
2,1,3,60,Arsenal,Liverpool,1.944129,1.465860,True,1617,NaN
3,1,3,0,Watford,Southampton,0.704200,0.796805,False,1617,NaN
4,1,2,0,Chelsea,West Ham United,2.243698,0.895471,True,1617,NaN
5,1,4,90,Hull City,Leicester City,0.494447,0.650832,True,1617,NaN
6,1,4,90,Hull City,Leicester City,0.494447,0.650832,True,1617,NaN
7,1,3,90,Middlesbrough,Stoke City,0.452793,0.718705,True,1617,NaN
8,1,1,0,Bournemouth,Manchester United,0.384921,1.983179,True,1617,NaN
9,1,3,69,Middlesbrough,Stoke City,0.452793,0.718705,True,1617,NaN


In [63]:
X.loc[:,cont_vars]

,minutes,relative_market_value_team,relative_market_value_opponent_team
0,0,0.895471,2.243698
1,15,1.057509,1.433690
2,60,1.944129,1.465860
3,0,0.704200,0.796805
4,0,2.243698,0.895471
5,90,0.494447,0.650832
6,90,0.494447,0.650832
7,90,0.452793,0.718705
8,0,0.384921,1.983179
9,69,0.452793,0.718705
